<a href="https://colab.research.google.com/github/mratanusarkar/Web-Scraping-tickertapeIN/blob/basic-scraper-colab/Web_Scraping_tickertapeIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scraping stock data from tickertape.in

**Input**: stock name in "https://www.tickertape.in/stocks/{stock-name}" <br>
**Output**: full stock data & predictions from tickertape in JSON/Py Dictionary format


## Import Packages

In [13]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

import time
from datetime import timedelta

## Request and Fetch the Webpage

Let's try with a sample stock name, say "TCS"

In [14]:
# hit "https://www.tickertape.in/stocks/tata-consultancy-services-TCS"
requests.get("https://www.tickertape.in/stocks/tata-consultancy-services-TCS")

<Response [200]>

In [15]:
# wow! no restriction for bots! no need of any headers!
response = requests.get("https://www.tickertape.in/stocks/tata-consultancy-services-TCS")
response.text[0:500]

'<!DOCTYPE html><html lang="en-US"><head><meta http-equiv="X-UA-Compatible" content="IE=edge"/><link rel="shortcut icon" href="/favicon/favicon.png"/><link rel="apple-touch-icon" href="/favicon/favicon-192x192.png"/><link rel="manifest" href="/manifest/manifest.json"/><style type="text/css">:root {--white: #ffffff; --font_primary: #535B62; --font_dark: #2f363f; --font_light: #81878c; --font_blue: #0088ea; --font_lighter: #a2a8ae; --brand_primary: #151e28; --brand_success: #28c39a; --brand_danger:'

In [100]:
# not required for this webpage, use if bot restrictions are added in future.

# google chrome browser's request header (to make it look like, we are making this request from a browser)
header = {
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"
}

# hit using the header
response = requests.get("https://www.tickertape.in/stocks/tata-consultancy-services-TCS", headers=header)
response.text[0:500]

'<!DOCTYPE html><html lang="en-US"><head><meta http-equiv="X-UA-Compatible" content="IE=edge"/><link rel="shortcut icon" href="/favicon/favicon.png"/><link rel="apple-touch-icon" href="/favicon/favicon-192x192.png"/><link rel="manifest" href="/manifest/manifest.json"/><style type="text/css">:root {--white: #ffffff; --font_primary: #535B62; --font_dark: #2f363f; --font_light: #81878c; --font_blue: #0088ea; --font_lighter: #a2a8ae; --brand_primary: #151e28; --brand_success: #28c39a; --brand_danger:'

## Pass the fetched webpage response to Beautiful Soup

In [101]:
# give the webpage to Beautiful Soup using parsers: "html.parser" or "lxml"
soup = BeautifulSoup(response.text, 'lxml')

## Let us try and extract some data from the soup

- we see the whole webpage and how the html dom structure is made
- on inspecting, we see that all our required info is mostly inside div blocks with unique class names
- few are inside span or h tags, but all blocks has classes
- Let us extract few important html dom blocks and see

### [1] Basic Company Information

In [29]:
# company name
htmlBlock = soup.find("h3", class_="security-name")
print(htmlBlock.prettify())

<h3 class="jsx-2903438179 security-name">
 Tata Consultancy Services Ltd
</h3>


In [30]:
# ticker name
htmlBlock = soup.find("span", class_="ticker")
print(htmlBlock.prettify())

<span class="jsx-2903438179 ticker text-teritiary font-medium">
 TCS
</span>


In [31]:
# current price
htmlBlock = soup.find("span", class_="current-price")
print(htmlBlock.prettify())

<span class="jsx-3168773259 current-price typography-h1 text-primary">
 3,685.65
</span>



In [102]:
# change absolute-value
htmlBlock = soup.find("span", class_="absolute-value")
print(htmlBlock.prettify())

<span class="jsx-3168773259 change absolute-value text-14 typography-body-medium-l up">
 <i class="jsx-3168773259 icon-Green-up">
 </i>
 0.04
 <!-- -->
 %
</span>



In [53]:
# change percentage-value
htmlBlock = soup.find("span", class_="percentage-value")
print(htmlBlock.prettify())

<span class="jsx-3168773259 change percentage-value text-14 up">
 (
 <!-- -->
 +
 <!-- -->
 1.50
 <!-- -->
 )
</span>


### [2] Investment Checklist

In [70]:
# checklist-item carousel-item
htmlBlock = soup.find("div", class_="carousel-item")
print(htmlBlock.prettify())

<div class="jsx-3083281824 checklist-item carousel-item selected">
 <div class="jsx-3228946760 jsx-482152645 commentary-item-root d-flex-row align-start justify-start " eventlabel="Clicked Checklist Item">
  <i class="jsx-3228946760 jsx-482152645 icon-mood icon-negative-comment text-24 mr12">
  </i>
  <div class="jsx-3228946760 jsx-482152645 content">
   <h4 class="jsx-3228946760 jsx-482152645 typography-body-medium-m text-primary">
    <span class="jsx-3228946760 jsx-482152645 relative no-select tooltip-holder">
     Intrinsic Value
     <div class="jsx-1503855875 tooltip-root sh-tooltip font-regular">
      Intrinsic value is the calculated value of the company and may differ from current stock price. If intrinsic value &gt; current price, price increase is expected in the future to reduce the gap and vice-versa
     </div>
    </span>
   </h4>
   <p class="jsx-3228946760 jsx-482152645 lh-138 text-13 text-secondary typography-body-regular-m commentary-desc">
    Current price is more

In [98]:
# get all keys and values

for item in htmlBlock.childGenerator():
  print(item.find("span", class_="tooltip-holder").contents[0], end=": ")
  print(item.find("i")['class'][3].split("-")[1])

Intrinsic Value: negative
ROE vs FD rates: positive
Dividend Returns: positive
Entry Point: positive
No Red Flags: positive


### [3] Price Chart

### [4] Key Metrics

### [5] Forecast & Ratings

In [104]:
# Forecast
htmlBlock = soup.find("div", class_="forecast-radial")
print(htmlBlock.prettify())

<div class="jsx-3770717616 forecast-radial">
 <div class="jsx-3770717616 radial-holder">
  <div class="rv-xy-plot rv-radial-chart" style="width:64px;height:64px">
   <svg class="rv-xy-plot__inner" height="64" width="64">
    <g class="rv-xy-plot__series rv-xy-plot__series--arc " opacity="1" pointer-events="all" transform="translate(32,32)">
     <path class="rv-xy-plot__series rv-xy-plot__series--arc-path rv-radial-chart__series--pie__slice " d="M-6.95776561860984,31.234428081789666A32,32,0,0,1,2.254340479449671e-14,-32L1.6907553595872534e-14,-24A24,24,0,0,0,-5.21832421395738,23.425821061342248Z" style="opacity:1;stroke:transparent;fill:rgba(129, 135, 140, 0.22)">
     </path>
     <path class="rv-xy-plot__series rv-xy-plot__series--arc-path rv-radial-chart__series--pie__slice " d="M1.959434878635765e-15,-32A32,32,0,1,1,-6.95776561860984,31.234428081789666L-5.21832421395738,23.425821061342248A24,24,0,1,0,1.4695761589768238e-15,-24Z" style="opacity:1;stroke:transparent;fill:#07d459">
  

In [115]:
print(htmlBlock.div.span.contents[0], htmlBlock.div.span.span.text)

53 %


In [111]:
print(htmlBlock.h4.text)

Analysts have suggested that investors can buy this stock
